In [85]:
import torch
from dataloading import MultitaskDataset
import time
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pickle
import zarr
from sklearn.model_selection import train_test_split

### Dataloading

In [127]:
def replace_y(x):
    mapping = {1: 0, 2: 1, 3: 2, 4: 3}
    return mapping.get(x, x)

def replace_d(x):
    mapping = {2: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7, 10: 8, 11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14}
    return mapping.get(x, x)

def loso(X,y,d):
    left_out_subject = 14
    idx = (d != left_out_subject)
    X_train = X[idx]
    y_train = y[idx]
    d_train = d[idx]

    # test data selecting just 14 subject
    idxt = (d == left_out_subject)
    X_test = X[idxt]
    y_test = y[idxt]
    d_test = d[idxt]


    return X_train, y_train, d_train, X_test, y_test, d_test

def data_loader():
    zarr_array = zarr.open("./dataset/chest_ECG_w60_mw60_ts256_cl2_cs1_fp[1.0].zarr/", mode="r")
    signal = zarr_array['raw_signal'][:]
    target_all = zarr_array['target'][:]
    subjects_all = zarr_array['subject'][:]

    SUBJECTS_IDS = list(range(2, 18))
    subjects = SUBJECTS_IDS[:]
    classes = [1, 2, 3, 4]

    subset_map = [
        idx
        for idx, i in enumerate(target_all)
        if i in classes and subjects_all[idx] in subjects
    ]

    idx = subset_map
    X = signal[idx]
    y = target_all[idx]
    d = subjects_all[idx]

    y_updated = np.vectorize(replace_y)(y)
    d_updated = np.vectorize(replace_d)(d)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_updated, test_size=0.33, random_state=42)
    
    
          
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = data_loader()

In [128]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [129]:
X_train = np.array(X_train_scaled).reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test = np.array(X_test_scaled).reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

In [130]:
y_train= np.array(y_train).reshape(y_train.shape[0], 1)
y_test=np.array(y_test).reshape(y_test.shape[0], 1)

In [131]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

In [132]:
dataset = TensorDataset(X_test,y_test )
test_loader = DataLoader(dataset)

In [133]:
y_train.shape

torch.Size([15061, 1])

### Model

In [134]:
class ECG_CNN(nn.Module):
    def __init__(self):
        super(ECG_CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool = nn.AdaptiveMaxPool1d(output_size=1)
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(64, 4)
    
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = x.view(-1, 64)
        x = self.fc1(x)
        return x

In [135]:
# initialize the model
model = ECG_CNN()

# set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Training

In [136]:
# train the model
num_epochs = 10
for epoch in range(num_epochs):
    # set model to training mode
    model.train()
    running_loss = 0.0
    for i in range(X_train.size(0)):
        # zero out the gradients
        optimizer.zero_grad()
        # forward pass
        outputs = model(X_train[i])
        # calculate loss
        loss = criterion(outputs, y_train[i])
        # backward pass
        loss.backward()
        # update parameters
        optimizer.step()
        running_loss += loss.item()
    print('Epoch [%d], Loss: %.7f' % (epoch+1, running_loss/X_train.size(0)))

Epoch [1], Loss: 1.3086
Epoch [2], Loss: 1.2623
Epoch [3], Loss: 1.2303
Epoch [4], Loss: 1.2121
Epoch [5], Loss: 1.1978
Epoch [6], Loss: 1.1857
Epoch [7], Loss: 1.1786
Epoch [8], Loss: 1.1696
Epoch [9], Loss: 1.1607
Epoch [10], Loss: 1.1530


### Model Evaluation

In [ ]:
model.eval()

total_correct = 0
total_pred = 0

with torch.no_grad():
    for X_test, y_test in test_loader:
        X_test, y_test = X_test, y_test
        
        outputs = model(X_test)
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        total_pred += y_test.size(0)
        total_correct += (predicted == y_test).sum().item()

accuracy = total_correct / total_pred
print("Accuracy: {:.2f}%".format(accuracy * 100))